In [7]:
#Import the appropriate module
import os


In [8]:
os.chdir('E:/Training files/ECC')

In [9]:
from ECCdataset import Ecc_DataSet
from spektral.data import BatchLoader
from spektral.layers import ECCConv, GlobalSumPool, GraphMasking
from tensorflow.keras.callbacks import ReduceLROnPlateau
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,concatenate,BatchNormalization,LayerNormalization,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from scipy import sparse as sp
from spektral.utils import pad_jagged_array
from spektral.utils.sparse import sp_matrix_to_sp_tensor

In [27]:
# Generate data sets.

Dataset = Ecc_DataSet(amount=None)
def load_dataset(Dataset):
    '''
    * Import the dataset and divide it into training, 
    * validation and test sets in a ratio of 8 : 1 : 1.
    *
    * Attributes
    * ----------
    * Dataset  : datasets to be divided
    * 
    * Returns
    * -------
    * dataset_tr, dataset_va,dataset_te : three scaled data sets
    '''
    
    idxs = np.random.permutation(len(Dataset))
    split_1 = int(0.8 * len(Dataset))
    split_2 = int(0.1 * len(Dataset))
    idx_tr, temp = np.split(idxs, [split_1])
    idx_va, idx_te = np.split(temp, [split_2])
    dataset_tr, dataset_va,dataset_te = Dataset[idx_tr], Dataset[idx_va],Dataset[idx_te]
    return dataset_tr, dataset_va,dataset_te

Loading Ecc_DataSet .
Reading SDF


100%|██████████████████████████████████████| 7246/7246 [00:26<00:00, 275.88it/s]


In [20]:
#build a model
class Ecc_model(Model):
    def __init__(self,ecc1=56,ecc2=16,ecc3=32,ds1 = 512,ds2 =512,ds3 = 32, dropout = 0.1):
        super().__init__()
        self.conv1 = ECCConv(ecc1, activation="relu")
        self.conv2 = ECCConv(ecc2, activation="relu")
        self.conv3 = ECCConv(ecc3, activation="relu")
        self.global_pool = GlobalSumPool()
        self.dropout = Dropout(rate=dropout)
        self.dense1 = Dense(ds1,activation='relu',kernel_regularizer = 'l2')
        self.dense2 = Dense(ds2,activation='relu',kernel_regularizer = 'l2')
        self.dense3 = Dense(ds3,activation='relu',kernel_regularizer = 'l2')
        self.dense4 = Dense(1)

    def call(self, inputs):
        x, a, e = inputs
        x = self.conv1([x, a, e])
        x = self.conv2([x, a, e])
        x = self.conv3([x, a, e])
        output = self.global_pool(x)
        output = self.dense1(output)
        output = self.dropout(output)
        output = self.dense2(output)
        output = self.dropout(output)
        output = self.dense3(output)
        output = self.dense4(output)
        return output

In [30]:
epochs =100
batch_size =8
learning_rate=0.001
idx_tr = []
with open('dataset_train.txt') as f :
    for i in f:
        idx_tr.append(int(i))
        
idx_va = []
with open('dataset_valid.txt') as f :
    for i in f:
        idx_va.append(int(i))
    
idx_te = []
with open('dataset_test.txt') as f :
    for i in f:
        idx_te.append(int(i))

dataset = Ecc_DataSet(amount=None)

dataset_tr, dataset_va,dataset_te = dataset[idx_tr], dataset[idx_va],dataset[idx_te]
loader_tr = BatchLoader(dataset_tr, batch_size=batch_size,epochs=epochs)
loader_va = BatchLoader(dataset_va, batch_size=batch_size,epochs=epochs)
model = Ecc_model()
optimizer = Adam(learning_rate)
model.compile(loss='mean_squared_error',optimizer=adam, metrics=['mean_absolute_error'])
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=1, epsilon=1e-4, mode='min',min_lr=0.00001)
history = model.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs,
          validation_data=loader_va.load(),validation_steps = loader_va.steps_per_epoch, 
          callbacks=[reduce_lr])


    


Loading Ecc_DataSet .
Reading SDF


100%|██████████████████████████████████████| 7246/7246 [00:26<00:00, 270.02it/s]


D:\Anaconda3\envs\spektral_2\lib\site-packages\spektral-1.0.8-py3.8.egg\spektral\data\utils.py:221: UserWarning: you are shuffling a 'Ecc_DataSet' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.


Epoch 1/100
725/725 [==============================] - 32s 43ms/step - loss: 6614.0757 - mean_absolute_error: 62.6095 - val_loss: 3520.7349 - val_mean_absolute_error: 46.9060
Epoch 2/100
725/725 [==============================] - 30s 42ms/step - loss: 3948.3972 - mean_absolute_error: 48.8085 - val_loss: 3886.4426 - val_mean_absolute_error: 48.2234
Epoch 3/100
725/725 [==============================] - 31s 42ms/step - loss: 3655.0198 - mean_absolute_error: 46.5933 - val_loss: 3276.5522 - val_mean_absolute_error: 42.6458
Epoch 4/100
725/725 [==============================] - 31s 42ms/step - loss: 3315.6028 - mean_absolute_error: 44.3282 - val_loss: 2763.8259 - val_mean_absolute_error: 39.9020
Epoch 5/100
725/725 [==============================] - 30s 42ms/step - loss: 3277.4307 - mean_absolute_error: 43.7830 - val_loss: 2620.6431 - val_mean_absolute_error: 40.4875
Epoch 6/100
725/725 [==============================] - 31s 42ms/step - loss: 3144.3271 - mean_absolute_error: 42.8364 - val_l